In [ ]:
from ZV_functions import *
import tensorflow as tf
from matplotlib import pyplot as plt
import matplotlib
import pandas as pd
matplotlib.rcParams.update({'font.size': 16})

In [ ]:
#base_dir='/eos/home-a/ahakimi/www/ZV_analysis'
base_dir='D:\\Travail\\ZV_analysis\\'
year='2018_SR_new'
cut='Boosted_SR'
vers=''

variables_list= ['Resolved_SR_pt1', 'Resolved_SR_pt2', 'Resolved_SR_eta1',
       'Resolved_SR_eta2', 'Resolved_SR_mll', 'Resolved_SR_Zepp_ll',
       'Resolved_SR_nFatJet', 'Resolved_SR_FatJet_pt', 'Resolved_SR_FatJeteta',
       'Resolved_SR_FatJet_softdropmass', 'Resolved_SR_FatJet_tau21',
       'Resolved_SR_Zlep_1', 'Resolved_SR_Zlep_2', 'Resolved_SR_category',
       'Resolved_SR_vbs_jet_pt1', 'Resolved_SR_vbs_jet_pt2',
       'Resolved_SR_vbs_jet_eta1', 'Resolved_SR_vbs_jet_eta2',
       'Resolved_SR_V_jet_pt1', 'Resolved_SR_V_jet_pt2',
       'Resolved_SR_V_jet_eta1', 'Resolved_SR_V_jet_eta2',
       'Resolved_SR_mjj_max', 'Resolved_SR_detajj_mjjmax',
       'Resolved_SR_dphijj_mjjmax', 'Resolved_SR_V_jet_mass']
if cut == 'Boosted_SR':
    features =['pt1', 'pt2', 'eta1',
           'eta2', 'mll',
           'FatJet_pt', 'FatJeteta',
           'Zlep_1', 'Zlep_2', 
           'vbs_jet_pt1', 'vbs_jet_pt2',
           'vbs_jet_eta1', 'vbs_jet_eta2',
           'mjj_max', 'detajj_mjjmax']
elif cut == 'Resolved_SR'
    features=['pt1', 'pt2', 'eta1',
           'eta2', 'mll',
           'Zlep_1', 'Zlep_2', 
           'vbs_jet_pt1', 'vbs_jet_pt2',
           'vbs_jet_eta1', 'vbs_jet_eta2',
           'V_jet_pt1', 'V_jet_pt2',
           'V_jet_eta1', 'V_jet_eta2',
           'mjj_max', 'detajj_mjjmax',
           'V_jet_mass']


In [ ]:

data_dir=base_dir+year+'\\'+cut+'\\data'+vers


input_list= features
inputs= [cut+'_'+i for i in input_list]



X= pd.DataFrame(np.load(data_dir+'\\X_{}_{}{}.npy'.format(year,cut,vers), allow_pickle=True), columns=variables_list)
X=X[inputs]
y= pd.DataFrame(np.load(data_dir+'\\y_{}_{}{}.npy'.format(year,cut,vers), allow_pickle=True), columns= ['signal','sample','group','weight_', 'w'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2, random_state=42)


session_conf = tf.ConfigProto(intra_op_parallelism_threads=12, inter_op_parallelism_threads=12)
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
k.tensorflow_backend.set_session(sess)

early_stop=5  #number of epochs without gain before stoping
predictions_NN={}
target={}


In [ ]:
X,y, variables_list=prep_data(year, cut, vers, base_dir) #

xlims=[(0,200),(0,150),(-3,3),(-3,3),(0,150),(0,5),(0,10),(0,900),(-5,5),(0,200),(0,1),(-1.5,1.5),(-1.5,1.5),(-1,1),
       (0,200),(0,200),(-5,5),(-5,5),
           (0,200),(0,100),(-5,5),(-5,5),(0,3000),(0,10),(0,4),(0,150)]

#plot_distrib(variables_list, xlims,base_dir, year, cut)
print('NUmber of events: {}, signal : {}, bkg: {}'.format(len(y),len(y[y['signal']==1]),len(y[y['signal']==0])))
print('SOW: {}, signal : {}, bkg: {}'.format(sum(y['weight_']),sum(y['weight_'][y['signal']==1]),sum(y['weight_'][y['signal']==0])))

In [ ]:
def build_dnn(base_dir, lr, l1, l2, dropout, sl1, sl2, sl3, sl4, sl5):
    from keras.layers.normalization import BatchNormalization

    SoB_thr=0.8
   
    
    NN_name='DNN_{}_{}_{}_{}_{}_lr{}_dp{}'.format(sl1,sl2,sl3,sl4,sl5,lr,dropout)
    reg=regularizers.L1L2(l1,l2)
    opt=Adam(lr=lr)
    #opt=SGD(lr=lr, momentum=0.99)


    DNN=Sequential()
    act_func=tfunc
    #,kernel_constraint=maxnorm(3)
    DNN.add(Dense(sl1,input_dim=len(inputs),activation=act_func,kernel_regularizer=reg))
    DNN.add(BatchNormalization())
    DNN.add(Dropout(rate=dropout))
    

    if sl2!=0:
        DNN.add(Dense(sl2,activation=act_func,kernel_regularizer=reg))
        DNN.add(BatchNormalization())
        DNN.add(Dropout(rate=dropout))
        
    if sl3!=0:
        DNN.add(Dense(sl3,activation=act_func,kernel_regularizer=reg))
        DNN.add(BatchNormalization())
        DNN.add(Dropout(rate=dropout))
        
    if sl4!=0:
        DNN.add(Dense(sl4,activation=act_func))
        DNN.add(BatchNormalization())
        DNN.add(Dropout(rate=dropout))
        
    if sl5!=0:
        DNN.add(Dense(sl5,activation=act_func))
        DNN.add(BatchNormalization())
        DNN.add(Dropout(rate=dropout))
        


    DNN.add(Dense(1,activation='sigmoid'))
    #DNN.add(BatchNormalization())
    #tp= tf.keras.metrics.TruePositives(thresholds =SoB_thr, name= 'tp')
    #fp= tf.keras.metrics.FalsePositives(thresholds =SoB_thr, name= 'fp')
    auc=tf.keras.metrics.AUC()
    DNN.compile(loss=loss,optimizer=opt ) #, metrics=[ auc]'auc [ auc, tp, fp]
    
    return DNN, NN_name

In [ ]:
import bayes_opt
import tensorflow as tf
from keras.layers import Dense, Dropout, LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.constraints import maxnorm
import keras.backend as k
from tensorflow.keras import regularizers
from sklearn.metrics import roc_curve, auc


def Score(roc_auc, roc_auc_train):
    alpha=1
    return roc_auc-alpha*(np.abs(roc_auc-roc_auc_train))
    #function to optimize in optimization
    
def Score_dnn(n_layer, size_layer, lr):
    lr=lr
    l1=0
    l2=0.001
    dropout=0.2
    n_layer=int(n_layer)
    size_layer=int(size_layer)
    sl1=size_layer
    sl2,sl3,sl4,sl5 = 0,0,0,0
    if n_layer >= 2:
        sl2=size_layer
    if n_layer >= 3:
        sl3=size_layer
    if n_layer >= 4:
        sl4=size_layer
    if n_layer >= 5:
        sl5=size_layer
    
    base_dir='D:\\Travail\\ZV_analysis\\'
    print('training network with [{}_{}_{}_{}_{}] neurons'.format(sl1,sl2,sl3,sl4,sl5))
    DNN, res_dir=build_dnn(base_dir, lr, l1, l2, dropout, sl1, sl2, sl3, sl4, sl5)
    
    es_callback = EarlyStopping(monitor='val_loss', patience=early_stop, restore_best_weights=True)
    history=DNN.fit(x=X_train, y=y_train['signal'].to_numpy(),
                    sample_weight=y_train['w'].to_numpy(), epochs=epochs, batch_size=batchsize, 
                    verbose=0, callbacks=[es_callback], 
                    validation_data=(X_test, y_test['signal'].to_numpy(),y_test['w'].to_numpy()))

    test_loss, test_acc = DNN.evaluate(X_test, y_test['signal'], verbose=0)
    #test = DNN.evaluate(X_test, y_test['signal'], verbose=0)
    #print(test)
    #print('\nTest accuracy:', test_acc)

    predictions_NN= DNN.predict(X_test)
    predictions_NN_train=DNN.predict(X_train)
    fpr, tpr, threshold = roc_curve(y_test['signal'].astype('int32'),predictions_NN, pos_label=1, sample_weight=y_test['w'])
    tpr.sort()
    fpr.sort()
    roc_auc =auc(fpr, tpr)
    fpr, tpr, threshold = roc_curve(y_train['signal'].astype('int32'),predictions_NN_train, pos_label=1, sample_weight=y_train['w'])
    tpr.sort()
    fpr.sort()
    roc_auc_train =auc(fpr, tpr)
    return Score(roc_auc, roc_auc_train)

In [ ]:
#optimize lr, l1,l2, dropout, sls

epochs=200
batchsize=256
tfunc='relu'
loss='binary_crossentropy'

pbounds={'n_layer': (1, 5),
         'size_layer': (20, 300),
         'lr':(1e-5, 1e-2),

      
        }

optimizer = bayes_opt.BayesianOptimization(
    f=Score_dnn,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [ ]:
optimizer.maximize(init_points=25, n_iter=40)
optimizer.max